In [1]:
# import packages we need
import requests as requests
import json
from datetime import datetime, timedelta, date
from pytz import timezone
import pandas as pd
import tweepy
import os
import psycopg2
from psycopg2 import extras

# # to run on heroku
# # Consumer keys and access tokens, used for OAuth
# songkick_api_key = os.environ['songkick_api_key']
# tw_consumer_key = os.environ["tw_consumer_key"]
# tw_consumer_secret = os.environ["tw_consumer_secret"]
# tw_access_token = os.environ["tw_access_token"]
# tw_access_token_secret = os.environ["tw_access_token_secret"]
# database_url = os.environ['HEROKU_POSTGRESQL_IVORY_URL']
# heroku_app_name =  os.environ['heroku_app_name']
# table_name =  os.environ['table_name']

# to run locally
keys={}
with open(os.path.abspath("keys.json"),"r") as f:
    keys = json.loads(f.read())
songkick_api_key = keys['songkick_api_key']
tw_consumer_key = keys["tw_consumer_key"]
tw_consumer_secret = keys["tw_consumer_secret"]
tw_access_token = keys["tw_access_token"]
tw_access_token_secret = keys["tw_access_token_secret"]
database_url = keys['HEROKU_POSTGRESQL_IVORY_URL']
heroku_app_name =  keys['heroku_app_name']
table_name =  keys['table_name']


# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(tw_consumer_key, tw_consumer_secret)
auth.set_access_token(tw_access_token, tw_access_token_secret)
 
# Creation of the actual interface, using authentication
api = tweepy.API(auth)

# our timezone
eastern = timezone('US/Eastern')

# this sends a tweet :)
def sendTweet(content):
    api.update_status(content)
    
# this gets one page of an artists history, up to 50 shows
def getArtistHistoryPage(artistId,page):
    
    url = "https://api.songkick.com/api/3.0/artists/" + str(artistId) + "/gigography.json"
    params = {'apikey': songkick_api_key, "page" : page, "per_page" : 50}
    resp = requests.get(url, params=params) 
    return json.loads(resp.text)

# this searches an artists full history and returns 
# true or false if they have ever played a show in dc
def wasArtistInCity(city, artistId):
    
    page = 1
    resp = getArtistHistoryPage(artistId, page)
    results = resp["resultsPage"]["results"]
    while (results != {}):
        events = results["event"]
        for event in events:
            if (city == event['location']['city']):
                return True

        page = page + 1
        resp = getArtistHistoryPage(artistId, page)
        results = resp["resultsPage"]["results"]
    
    return False

# this gets one page of the upcoming shows in DC, up to 50 shows
def getUpcomingShowsPage(metroId, minDate, maxDate, page):
    
    url = "https://api.songkick.com/api/3.0/metro_areas/" + metroId + "/calendar.json"
    params = {'apikey': songkick_api_key, "min_date" : minDate, "max_date" : maxDate, "page" : page, "per_page" : 50}
    resp = requests.get(url, params=params) 
    return json.loads(resp.text)

# this gets all of the upcoming shows in DC
def getUpcomingShows(daysAhead, metroId):
   
    artistIds = []
    artistNames = []
    artistUrls = []
    billings = []
    billingIndexes = []
    eventIds = []
    eventTypes = []
    eventUrls = []
    eventDates = []
    venueIds = []
    venueNames = []
    locationCities = []
    concertTimes = []
    
    page = 1
    minDate = str(date.today())
    maxDate = str(date.today() + timedelta(daysAhead))
    resp = getUpcomingShowsPage(metroId, minDate,maxDate, page)
    results = resp["resultsPage"]["results"]
    
    while (results != {}):
        events = results["event"]
    
        for event in events:
            for performance in event["performance"]:
                artist = performance["artist"]
                if ("PRIVATE EVENT" not in artist["displayName"]):
                    billings.append(performance["billing"])
                    billingIndexes.append(performance["billingIndex"])
                    artistIds.append(artist["id"])
                    artistNames.append(artist["displayName"])
                    artistUrls.append(artist["uri"])
                    eventIds.append(event['id']) 
                    eventTypes.append(event['type'])
                    eventUrls.append(event['uri'])
                    eventDates.append(event['start']['date'])
                    venueIds.append(event['venue']['id'])
                    venueNames.append(event['venue']['displayName'])
                    locationCities.append(event['location']['city']) 
                    concertTimes.append(event["start"]["datetime"])
 
        page = page + 1
        resp = getUpcomingShowsPage(metroId, minDate,maxDate, page)
        results = resp["resultsPage"]["results"]
    
    data = pd.DataFrame(
            {
                "artistId" : artistIds,
                "artistName" : artistNames,
                "artistUrl" : artistUrls,
                "billing" : billings,
                "billingIndex" : billingIndexes,
                "eventId" : eventIds,
                "eventType" : eventTypes,
                "eventUrl" : eventUrls,
                "eventDate" : eventDates,
                "venueId" : venueIds,
                "venueName" : venueNames,
                "locationCity" : locationCities,
                "concertTime" : concertTimes
            })
    
    return data

# this adds the correct ordinal to the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

# this runs every hour, it sends the first tweet that is qualified if there is one
def sendNextTweet(toTweet):
    if (len(toTweet) != 0):
        # maybe add a check now or somekind of resorting logic to make sure show has not already happened
        
        timeNow = datetime.now(eastern)
        
        didWeTweet = False
        
        for index,row in toTweet.iterrows():
            print(row['artistName'])
            print(row['concertTime'])
            print(row['tweeted'])
            print("---------")
            # did we not already tweet something this hour and did it not already happen and did we not already tweet it?
            if ((didWeTweet == False) and (row['concertTime'] > timeNow) and (row['tweeted'] == 0)):
                didWeTweet = True
                thisTweet = row['content']
                print("****")
                print(thisTweet)
                print("****")
#                     sendTweet(thisTweet)
                toTweet.loc[index,'tweeted'] = 1
        print("#####################")
    else:
        print("nothing to send")
        
    return toTweet

def writeTable(toTweet):
    toTweet = toTweet.sort_values(by=['concertTime'], ascending=True)
    if len(toTweet) > 0:
        conn = psycopg2.connect(database_url)
        cursor = conn.cursor()
        df_columns = list(toTweet)
        columns = ",".join(df_columns)
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
        insert_stmt = "INSERT INTO {} ({}) {}".format(table_name,columns,values)

        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, toTweet.values)
        conn.commit()
        cur.close()
        conn.close()

def clearTable():
    clear_table = "DELETE FROM " + table_name + ";"
    conn = psycopg2.connect(database_url)
    cursor = conn.cursor()
    cursor.execute(clear_table)
    conn.commit() # <--- makes sure the change is shown in the database
    conn.close()
    cursor.close()
    
def readTable():
    conn = psycopg2.connect(database_url)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM " + table_name) # <--- makes sure the change is shown in the database
    conn.commit()
    toTweet = cursor.fetchall()
    conn.close()
    cursor.close()
    
    toTweet = pd.DataFrame(toTweet,columns=["artistId","artistName","concertTime","content","eventDate","billingIndex","tweeted"])
    toTweet['concertTime'] = toTweet['concertTime'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S+00"))
    toTweet['concertTime'] = toTweet['concertTime'].apply(lambda x: x - timedelta(hours=(5)) + timedelta(minutes=(4)))
    toTweet['concertTime'] = toTweet['concertTime'].apply(lambda x: x.replace(tzinfo=eastern))
    toTweet = toTweet.sort_values(by=['concertTime'], ascending=True)
    return toTweet

# this runs once a day, it finds new artists in the area
def runBot(days, cityName, cityId, artistsWhoPlayedInDC):
        
    upcomingShows = getUpcomingShows(days,cityId)
    upcomingShows = upcomingShows[upcomingShows["locationCity"] == cityName]
    upcomingShows.drop_duplicates(subset ="artistId", inplace = True) 
    
    contents = []
    artistIds = []
    artistNames = []
    artistNames = []
    concertTimes = []
    eventDates = []
    artistUrls = []
    billingIndexes = []
    tweetedes = []
                    
    for index, row in upcomingShows.iterrows():
        artistId = row['artistId']
        eventDate = row['eventDate']
        concertTime = row['concertTime']
        venueName = row['venueName']
        eventUrl = row['eventUrl']
        artistName = row['artistName']
        billingIndex = row['billingIndex']
        if artistId not in artistsWhoPlayedInDC:
            if not wasArtistInCity(cityName, artistId):
                
                # make the tweet string
                dateString = datetime.strptime(eventDate, "%Y-%m-%d").strftime("%B") + " " + ordinal(datetime.strptime(eventDate, "%Y-%m-%d").day)
                content = (str(artistName) + " is playing their first show in DC!")
                if (billingIndex == 1):
                    content = content + " They are headlining at " + venueName + " on " + dateString + " " + eventUrl
                else:
                    content = content + " They are opening at " + venueName + " on " + dateString + " " + eventUrl   
                    
                # fix times that are null with noon of the day of cencert so it goes first
                if (concertTime == None):
                    concertTime = datetime.strptime(eventDate,"%Y-%m-%d")
                    concertTime = concertTime.replace(tzinfo=eastern)
                    concertTime = concertTime + timedelta(hours=(12))
                else:
                    concertTime = datetime.strptime(concertTime,"%Y-%m-%dT%H:%M:%S%z")
                    concertTime = concertTime.replace(tzinfo=eastern)
                
                # add one second per billing index
                concertTime = concertTime - timedelta(seconds=(billingIndex - 1))
                
                # add values to lists to make dataframe later
                artistIds.append(artistId)
                artistNames.append(artistName)
                contents.append(content)
                concertTimes.append(concertTime)
                eventDates.append(eventDate)
                billingIndexes.append(billingIndex)
                tweetedes.append(0)
            
    toTweetNew = pd.DataFrame(
            {
                "artistId" : artistIds,
                "artistName" : artistNames,
                "concertTime" : concertTimes,
                "content" : contents,
                "eventDate" : eventDates,
                "billingIndex" : billingIndexes,
                "tweeted" : tweetedes
            })
    return toTweetNew

def onceADay():
    toTweet = readTable()
    artistsWhoPlayedInDC = toTweet['artistId']
    cityName = "Washington, DC, US"
    cityId = "1409"
    days = 1
    toTweetNew  = runBot(days,cityName,cityId,artistsWhoPlayedInDC)

    # make sure we don't already have a artist
    toTweetNew = toTweetNew[~toTweetNew['artistId'].isin(artistsWhoPlayedInDC)]
    # combine results
    toTweet = pd.concat([toTweet, toTweetNew], ignore_index=True)
    # remove shows that are very old
    twoWeeksAgo = datetime.now(eastern) - timedelta(weeks=(2))
    toTweet = toTweetNew[toTweetNew['concertTime'] > twoWeeksAgo]

    # sort by time
    toTweet = toTweet.sort_values(by=['concertTime'], ascending=True)

    # clear
    clearTable()
    
    # upload
    writeTable(toTweet)
    print("got artists")
    
def everyHour():
    toTweet = readTable()
    print("Start sending new tweet now")
    toTweet = sendNextTweet(toTweet)
    clearTable()
    writeTable(toTweet)
    

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

clearTable()
# def timed_job():
#     onceADay()

onceADay()

def scheduled_job():
    everyHour()


scheduler = BlockingScheduler()
# scheduler.add_job(timed_job, 'cron', hour=8)
scheduler.add_job(scheduled_job, 'interval', seconds=10)
scheduler.start()
print("start script")

got artists
Start sending new tweet now
Ingrid Andress
2019-12-04 12:00:00-05:00
0
---------
The Offbrands
2019-12-04 12:00:00-05:00
0
---------
Say Sue Me
2019-12-04 19:00:00-05:00
0
---------
roselove
2019-12-04 19:00:00-05:00
0
---------
Caracara
2019-12-04 19:29:59-05:00
0
---------
Neil Hilborn
2019-12-04 19:30:00-05:00
0
---------
Haley Fohr
2019-12-05 11:59:59-05:00
0
---------
****
Haley Fohr is playing their first show in DC! They are opening at Hirshhorn Museum and Sculpture Garden on December 5th http://www.songkick.com/concerts/39283548-clarice-jensen-at-hirshhorn-museum-and-sculpture-garden?utm_source=59162&utm_medium=partner
****
Clarice Jensen
2019-12-05 12:00:00-05:00
0
---------
Another Sky
2019-12-05 12:00:00-05:00
0
---------
New Order of Existence
2019-12-05 18:59:58-05:00
0
---------
Nate Brown
2019-12-05 19:29:59-05:00
0
---------
Kicking Sunrise
2019-12-05 19:59:59-05:00
0
---------
So Sensitive
2019-12-05 19:59:59-05:00
0
---------
Chris Renzema
2019-12-05 20:00

Start sending new tweet now
Ingrid Andress
2019-12-04 12:32:00-05:00
0
---------
The Offbrands
2019-12-04 12:32:00-05:00
0
---------
Say Sue Me
2019-12-04 19:32:00-05:00
0
---------
roselove
2019-12-04 19:32:00-05:00
0
---------
Caracara
2019-12-04 20:01:59-05:00
0
---------
Neil Hilborn
2019-12-04 20:02:00-05:00
0
---------
Haley Fohr
2019-12-05 12:31:59-05:00
1
---------
Clarice Jensen
2019-12-05 12:32:00-05:00
1
---------
Another Sky
2019-12-05 12:32:00-05:00
1
---------
New Order of Existence
2019-12-05 19:31:58-05:00
1
---------
Nate Brown
2019-12-05 20:01:59-05:00
1
---------
Kicking Sunrise
2019-12-05 20:31:59-05:00
1
---------
So Sensitive
2019-12-05 20:31:59-05:00
1
---------
Chris Renzema
2019-12-05 20:32:00-05:00
1
---------
Nathan Brown
2019-12-05 22:01:58-05:00
0
---------
****
Nathan Brown is playing their first show in DC! They are opening at Twins Jazz on December 5th http://www.songkick.com/concerts/39242045-ron-jackson-at-twins-jazz?utm_source=59162&utm_medium=partner

Start sending new tweet now
Ingrid Andress
2019-12-04 13:12:00-05:00
0
---------
The Offbrands
2019-12-04 13:12:00-05:00
0
---------
Say Sue Me
2019-12-04 20:12:00-05:00
0
---------
roselove
2019-12-04 20:12:00-05:00
0
---------
Caracara
2019-12-04 20:41:59-05:00
0
---------
Neil Hilborn
2019-12-04 20:42:00-05:00
0
---------
Haley Fohr
2019-12-05 13:11:59-05:00
1
---------
Clarice Jensen
2019-12-05 13:12:00-05:00
1
---------
Another Sky
2019-12-05 13:12:00-05:00
1
---------
New Order of Existence
2019-12-05 20:11:58-05:00
1
---------
Nate Brown
2019-12-05 20:41:59-05:00
1
---------
Kicking Sunrise
2019-12-05 21:11:59-05:00
1
---------
So Sensitive
2019-12-05 21:11:59-05:00
1
---------
Chris Renzema
2019-12-05 21:12:00-05:00
1
---------
Nathan Brown
2019-12-05 22:41:58-05:00
1
---------
#####################
Start sending new tweet now
Ingrid Andress
2019-12-04 13:16:00-05:00
0
---------
The Offbrands
2019-12-04 13:16:00-05:00
0
---------
Say Sue Me
2019-12-04 20:16:00-05:00
0
---------

Start sending new tweet now
Ingrid Andress
2019-12-04 13:56:00-05:00
0
---------
The Offbrands
2019-12-04 13:56:00-05:00
0
---------
Say Sue Me
2019-12-04 20:56:00-05:00
0
---------
roselove
2019-12-04 20:56:00-05:00
0
---------
Caracara
2019-12-04 21:25:59-05:00
0
---------
Neil Hilborn
2019-12-04 21:26:00-05:00
0
---------
Haley Fohr
2019-12-05 13:55:59-05:00
1
---------
Clarice Jensen
2019-12-05 13:56:00-05:00
1
---------
Another Sky
2019-12-05 13:56:00-05:00
1
---------
New Order of Existence
2019-12-05 20:55:58-05:00
1
---------
Nate Brown
2019-12-05 21:25:59-05:00
1
---------
Kicking Sunrise
2019-12-05 21:55:59-05:00
1
---------
So Sensitive
2019-12-05 21:55:59-05:00
1
---------
Chris Renzema
2019-12-05 21:56:00-05:00
1
---------
Nathan Brown
2019-12-05 23:25:58-05:00
1
---------
#####################
Start sending new tweet now
Ingrid Andress
2019-12-04 14:00:00-05:00
0
---------
The Offbrands
2019-12-04 14:00:00-05:00
0
---------
Say Sue Me
2019-12-04 21:00:00-05:00
0
---------

Start sending new tweet now
Ingrid Andress
2019-12-04 14:40:00-05:00
0
---------
The Offbrands
2019-12-04 14:40:00-05:00
0
---------
Say Sue Me
2019-12-04 21:40:00-05:00
0
---------
roselove
2019-12-04 21:40:00-05:00
0
---------
Caracara
2019-12-04 22:09:59-05:00
0
---------
Neil Hilborn
2019-12-04 22:10:00-05:00
0
---------
Haley Fohr
2019-12-05 14:39:59-05:00
1
---------
Clarice Jensen
2019-12-05 14:40:00-05:00
1
---------
Another Sky
2019-12-05 14:40:00-05:00
1
---------
New Order of Existence
2019-12-05 21:39:58-05:00
1
---------
Nate Brown
2019-12-05 22:09:59-05:00
1
---------
Kicking Sunrise
2019-12-05 22:39:59-05:00
1
---------
So Sensitive
2019-12-05 22:39:59-05:00
1
---------
Chris Renzema
2019-12-05 22:40:00-05:00
1
---------
Nathan Brown
2019-12-06 00:09:58-05:00
1
---------
#####################
Start sending new tweet now
Ingrid Andress
2019-12-04 14:44:00-05:00
0
---------
The Offbrands
2019-12-04 14:44:00-05:00
0
---------
Say Sue Me
2019-12-04 21:44:00-05:00
0
---------

Start sending new tweet now
Ingrid Andress
2019-12-04 15:24:00-05:00
0
---------
The Offbrands
2019-12-04 15:24:00-05:00
0
---------
Say Sue Me
2019-12-04 22:24:00-05:00
0
---------
roselove
2019-12-04 22:24:00-05:00
0
---------
Caracara
2019-12-04 22:53:59-05:00
0
---------
Neil Hilborn
2019-12-04 22:54:00-05:00
0
---------
Haley Fohr
2019-12-05 15:23:59-05:00
1
---------
Clarice Jensen
2019-12-05 15:24:00-05:00
1
---------
Another Sky
2019-12-05 15:24:00-05:00
1
---------
New Order of Existence
2019-12-05 22:23:58-05:00
1
---------
Nate Brown
2019-12-05 22:53:59-05:00
1
---------
Kicking Sunrise
2019-12-05 23:23:59-05:00
1
---------
So Sensitive
2019-12-05 23:23:59-05:00
1
---------
Chris Renzema
2019-12-05 23:24:00-05:00
1
---------
Nathan Brown
2019-12-06 00:53:58-05:00
1
---------
#####################
Start sending new tweet now
Ingrid Andress
2019-12-04 15:28:00-05:00
0
---------
The Offbrands
2019-12-04 15:28:00-05:00
0
---------
Say Sue Me
2019-12-04 22:28:00-05:00
0
---------

Start sending new tweet now
Ingrid Andress
2019-12-04 16:04:00-05:00
0
---------
The Offbrands
2019-12-04 16:04:00-05:00
0
---------
Say Sue Me
2019-12-04 23:04:00-05:00
1
---------
roselove
2019-12-04 23:04:00-05:00
0
---------
****
roselove is playing their first show in DC! They are headlining at Slash Run on December 4th http://www.songkick.com/concerts/39274091-roselove-at-slash-run?utm_source=59162&utm_medium=partner
****
Caracara
2019-12-04 23:33:59-05:00
1
---------
Neil Hilborn
2019-12-04 23:34:00-05:00
1
---------
Haley Fohr
2019-12-05 16:03:59-05:00
1
---------
Clarice Jensen
2019-12-05 16:04:00-05:00
1
---------
Another Sky
2019-12-05 16:04:00-05:00
1
---------
New Order of Existence
2019-12-05 23:03:58-05:00
1
---------
Nate Brown
2019-12-05 23:33:59-05:00
1
---------
Kicking Sunrise
2019-12-06 00:03:59-05:00
1
---------
So Sensitive
2019-12-06 00:03:59-05:00
1
---------
Chris Renzema
2019-12-06 00:04:00-05:00
1
---------
Nathan Brown
2019-12-06 01:33:58-05:00
1
---------


Start sending new tweet now
Ingrid Andress
2019-12-04 16:44:00-05:00
0
---------
The Offbrands
2019-12-04 16:44:00-05:00
0
---------
Say Sue Me
2019-12-04 23:44:00-05:00
1
---------
roselove
2019-12-04 23:44:00-05:00
1
---------
Caracara
2019-12-05 00:13:59-05:00
1
---------
Neil Hilborn
2019-12-05 00:14:00-05:00
1
---------
Haley Fohr
2019-12-05 16:43:59-05:00
1
---------
Clarice Jensen
2019-12-05 16:44:00-05:00
1
---------
Another Sky
2019-12-05 16:44:00-05:00
1
---------
New Order of Existence
2019-12-05 23:43:58-05:00
1
---------
Nate Brown
2019-12-06 00:13:59-05:00
1
---------
Kicking Sunrise
2019-12-06 00:43:59-05:00
1
---------
So Sensitive
2019-12-06 00:43:59-05:00
1
---------
Chris Renzema
2019-12-06 00:44:00-05:00
1
---------
Nathan Brown
2019-12-06 02:13:58-05:00
1
---------
#####################
Start sending new tweet now
Ingrid Andress
2019-12-04 16:48:00-05:00
0
---------
The Offbrands
2019-12-04 16:48:00-05:00
0
---------
Say Sue Me
2019-12-04 23:48:00-05:00
1
---------

Start sending new tweet now
Ingrid Andress
2019-12-04 17:28:00-05:00
0
---------
The Offbrands
2019-12-04 17:28:00-05:00
0
---------
Say Sue Me
2019-12-05 00:28:00-05:00
1
---------
roselove
2019-12-05 00:28:00-05:00
1
---------
Caracara
2019-12-05 00:57:59-05:00
1
---------
Neil Hilborn
2019-12-05 00:58:00-05:00
1
---------
Haley Fohr
2019-12-05 17:27:59-05:00
1
---------
Clarice Jensen
2019-12-05 17:28:00-05:00
1
---------
Another Sky
2019-12-05 17:28:00-05:00
1
---------
New Order of Existence
2019-12-06 00:27:58-05:00
1
---------
Nate Brown
2019-12-06 00:57:59-05:00
1
---------
Kicking Sunrise
2019-12-06 01:27:59-05:00
1
---------
So Sensitive
2019-12-06 01:27:59-05:00
1
---------
Chris Renzema
2019-12-06 01:28:00-05:00
1
---------
Nathan Brown
2019-12-06 02:57:58-05:00
1
---------
#####################
Start sending new tweet now
Ingrid Andress
2019-12-04 17:32:00-05:00
0
---------
The Offbrands
2019-12-04 17:32:00-05:00
0
---------
Say Sue Me
2019-12-05 00:32:00-05:00
1
---------

In [ ]:
# create table

import subprocess, psycopg2
import sql

create_table = "CREATE TABLE " + table_name + " (\
    artistId int,\
    artistName varchar(255),\
    concertTime varchar(255),\
    content varchar(1000),\
    eventDate varchar(255),\
    billingIndex int,\
    tweeted int\
);"
conn = psycopg2.connect(database_url)
cursor = conn.cursor()
cursor.execute(create_table)
conn.commit() # <--- makes sure the change is shown in the database
conn.close()
cursor.close()

In [ ]:
# drop table

import subprocess, psycopg2
import sql

create_table = "DROP TABLE " + table_name + ";"
conn = psycopg2.connect(database_url)
cursor = conn.cursor()
cursor.execute(create_table)
conn.commit() # <--- makes sure the change is shown in the database
conn.close()
cursor.close()